# Imports

In [1]:
from pyabc import (ABCSMC, Distribution, RV,
                   History, Model,
                   ModelResult, MedianEpsilon)
#from pyabc.populationstrategy import AdaptivePopulationSize
#from pyabc.distance_functions import PNormDistance, WeightedPNormDistance
from pyabc.transition import MultivariateNormalTransition
from pyabc.visualization import plot_kde_matrix
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
%config Inline.Backend.figure_format = 'retina'
import os
import tempfile
import math
import pandas as pd
import scipy as sp
import numpy as np
import subprocess
from io import BytesIO

from pyabc_custom import MyokitSimulation
from prangle_distance import PrangleDistance, PrangleEpsilon, PranglePopulationSize

In [2]:
import logging
logging.basicConfig()
logging.getLogger().setLevel(logging.DEBUG)

# Get experimental measurements

In [3]:
myokit_python = ("/Users/charles/miniconda3/envs" +
                 "/ion_channel_ABC/bin/python")
args = [myokit_python, "get_measurements.py"]
args.append('icat')
re = subprocess.run(args, stdout=subprocess.PIPE)
measurements = pd.read_table(BytesIO(re.stdout),
                             delim_whitespace=True,
                             header=0, index_col=False)
obs = measurements.to_dict()['y']

# Set limits and generate uniform initial priors

In [4]:
limits = dict(g_CaT=(0, 2),
              E_CaT=(0, 50),
              p1=(0, 100),
              p2=(0, 10),
              p3=(0, 2), # increased from (0, 1) to (0, 2) after initial run
              p4=(0, 10),
              p5=(0, 0.1),
              p6=(0, 200),
              q1=(0, 100),
              q2=(0, 10),
              q3=(0, 10),
              q4=(0, 100),
              q5=(0, 0.1),
              q6=(0, 100))
prior = Distribution(**{key: RV("uniform", a, b - a)
                        for key, (a,b) in limits.items()})

In [5]:
prior.rvs()

<Parameter 'g_CaT': 0.04385971271632805, 'E_CaT': 49.1078801590994, 'p1': 43.54016582015644, 'p2': 7.656941567851447, 'p3': 0.7864805630327838, 'p4': 3.373327422437603, 'p5': 0.08794036782079726, 'p6': 7.651024702785469, 'q1': 54.715796335540205, 'q2': 2.9973046396201664, 'q3': 4.63096068245509, 'q4': 55.65336865858592, 'q5': 0.037427917675135494, 'q6': 68.94704945362261>

# Define distance function and simulation model

In [6]:
distance = PrangleDistance(alpha=0.5)

In [7]:
population = PranglePopulationSize(500, 0.5, mean_cv=0.5, 
                 min_population_size=10,
                 max_population_size=100)

In [8]:
myokit_model = MyokitSimulation('icat')

In [9]:
def identity(x):
    return x
result = myokit_model.accept(prior.rvs(),
                             identity,
                             lambda x: distance(x, obs),
                             {0: float('inf')})

In [10]:
result.distance

39.41224874393543

# Define ABC parameters and initialize database

In [11]:
db_path = ("sqlite:///" +
           os.path.join(tempfile.gettempdir(), "icat_hl-1.db"))

In [12]:
abc = ABCSMC(models=myokit_model,
             parameter_priors=prior,
             distance_function=distance,
             population_size=population,
             eps=PrangleEpsilon())

DEBUG:Epsilon:init prangle_epsilon initial_epsilon=inf, alpha=0.5


In [13]:
abc_id = abc.new(db_path, obs)

DEBUG:History:Database access through "_pre_calculate_id"
INFO:Epsilon:initial epsilon is inf
DEBUG:History:Database access through "store_initial_data"
DEBUG:git.cmd:Popen(['git', 'version'], cwd=/Users/charles/projects/ion-channel-ABC, universal_newlines=False, shell=None)
DEBUG:git.cmd:Popen(['git', 'version'], cwd=/Users/charles/projects/ion-channel-ABC, universal_newlines=False, shell=None)
DEBUG:git.cmd:Popen(['git', 'cat-file', '--batch-check'], cwd=/Users/charles/projects/ion-channel-ABC, universal_newlines=False, shell=None)
INFO:History:Start <ABCSMC(id=36, start_time=2018-05-23 11:51:14.788959, end_time=None)>


In [ ]:
history = abc.run(minimum_epsilon=0, max_nr_populations=10)

DEBUG:History:Database access through "max_t"
INFO:ABC:t:0 M eps:[inf]
DEBUG:History:Database access through "max_t"
DEBUG:History:Database access through "get_model_probabilities"
DEBUG:ABC:now submitting population 0
DEBUG:ABC:M population size: 1000


> /Users/charles/miniconda3/envs/pyabc/lib/python3.6/site-packages/pyabc-0.8.21-py3.6.egg/pyabc/smc.py(615)run()
-> sample = self.sampler.sample_until_n_accepted(


Initial run with reduced population to identify unconstrainable parameters.

In [ ]:
abc = ABCSMC(models=MyokitSimulation(),
             parameter_priors=prior,
             distance_function=distance,
             population_size=500,
             eps=MedianEpsilon(100, median_multiplier=1.0))
abc_id = abc.new(db_path, obs)

In [ ]:
history_init = abc.run(max_nr_populations=10, minimum_epsilon=0.01)

In [ ]:
from visualization_custom import plot_sim_results
plot_sim_results(history_init, "icat", n_samples=100, obs=measurements)

In [ ]:
df, w = history_init.get_distribution(m=0)
plot_kde_matrix(df, w, limits=limits)

In [ ]:
history_init = abc.run(max_nr_populations=20, minimum_epsilon=0.01)

In [ ]:
from visualization_custom import plot_sim_results
plot_sim_results(history_init, "icat", n_samples=100, obs=measurements)

In [ ]:
df, w = history_init.get_distribution(m=0)
plot_kde_matrix(df, w, limits=limits)

Reduce parameters varied and run with adaptive population size.

In [ ]:
abc = ABCSMC(models=MyokitSimulation(),
             parameter_priors=prior,
             distance_function=distance,
             population_size=AdaptivePopulationSize(
                 5000, 0.5, 
                 min_population_size=500,
                 max_population_size=10000),
            eps=MedianEpsilon(100, median_multiplier=1.0))
abc_id = abc.new(db_path, obs)

In [ ]:
history = abc.run(max_nr_populations=20, minimum_epsilon=0.5)